<a href="https://colab.research.google.com/github/DJNahata/Style-GANS-Practice/blob/master/Mapping_Network_in_NNabla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Installing Nnabla
!pip install nnabla

     |████████████████████████████████| 14.2MB 232kB/s 
     |████████████████████████████████| 4.8MB 34.2MB/s 


In [0]:
# Importing the Pre-Trained weights as the List.. using pickle
# Dense Layer Weights
import pickle
with open("G_mapping_weigh.txt", "rb") as fp:   
   G_mapping_weights = pickle.load(fp)

In [0]:
# Bias Layer Weights
with open("G_mapping_bi.txt", "rb") as fp:   
   G_mapping_bias = pickle.load(fp)

In [0]:
# Latent Input Vector z which is kept same
with open("latent_z.txt", 'rb') as fp:
   latent_z = pickle.load(fp)

In [0]:
import numpy as np
import nnabla as nn
import nnabla.parametric_functions as PF
import nnabla.functions as F
import nnabla.solvers as S
import nnabla.logger as logger
import nnabla.utils.save as save
from nnabla.parameter import get_parameter_or_create
from nnabla.ext_utils import get_extension_context
from nnabla.parametric_functions import parametric_function_api
import nnabla.initializer as I
nn.set_auto_forward(True)
print("DJ1")

2020-04-22 06:54:12,475 [nnabla][INFO]: Initializing CPU extension...


DJ1


In [0]:
def get_weight(he_std, gain = np.sqrt(2), use_wscale = False, lrmul = 1):
    if use_wscale:
        init_std = 1.0 / lrmul
        runtime_coeff = he_std * lrmul
    else: 
        init_std = he_std / lrmul
        runtime_coeff = lrmul

    w_init = I.NormalInitializer(sigma = init_std)   #I.NormalInitializer function is used with sigma argument as the standard deviation, so as to get the weights of these variables 
    return w_init, runtime_coeff

In [0]:
from __future__ import absolute_import
from six.moves import range
print("DJ1")

import functools
# Globally prepared params.
w = nn.parameter.get_parameter_or_create('BLURRING_FILTER', shape=(3,3), need_grad=False)  #why and how is it used for the parameter
w.d = np.array([[1., 2., 1.], [2., 4., 2.], [1., 2., 1.]])   

print('')


def lerp(a, b, t):
    return ((a + (b - a) * t))

def lerp_clip(a, b, t):
    return (a + (b - a) * F.clip_by_value(t, 0.0, 1.0))


#Computing the mean across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
#--------------------
def compute_channel_mean(x, keepdims = True):
    ch_mean = F.mean(x, axis = [2,3], keepdims = keepdims)
    return ch_mean

#--------------------


#-------------------------------
#Computing the standard deviation across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
def compute_channel_std(x, keepdims = True):
    x1 = compute_channel_mean(x)
    var = F.mean(F.pow_scalar((x - x1), 2.0), axis = [2,3], keepdims = keepdims)
    std = F.pow_scalar(var, 0.5)
    return std

#-------------------------------


#---------------------------------------------------
#Adaptive Instance Normalization to be used
def adain(x, y):
    y_s, y_b = y
    ch1_mean = compute_channel_mean(x)
    ch1_std = compute_channel_std(x)
    x = (x - ch1_mean) / ch1_std
    a1 = (y_s * x) + y_b
    return a1 

#-------------------------------


#New blur function to be used ------------------------------------------
def blur2d(x, f=[1,2,1], normalize = True, flip = False, stride = 1):
    ## assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(stride, int) and stride >= 1

    f = np.array(f, dtype = np.float32)

    if f.ndim == 1:
        f = f[:, np.newaxis] * f[np.newaxis, :]

    if flip:
        f = f[::-1, ::-1]

    if normalize:
        f = f / np.sum(f)

    f = f[ np.newaxis, :, :]
    print("F.shape is:", f.shape)
    f = np.tile(f, [int(x.shape[1]), 1, 1])
    print("F.shape is:",f.shape)
    if f.shape == (1, 1) and f[0,0] == 1:
        return x

    #orig_dtype = x.dtype
    strides = (stride, stride) 
    w = nn.Variable.from_numpy_array(f)
    print(w.shape)
    padding = (w.shape[1] // 2, w.shape[2] // 2)

    y = F.depthwise_convolution(x, w, pad = padding, stride = strides)
    return y

DJ1



In [0]:
#Upscale operation function to be used 
def upscale2d(x, factor = 2, gain = 1):
    # to check whether the shape of the x is 4 or not
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1 

    if gain!=1:
        x = x * gain
    
    if factor == 1:
        return x

    s = x.shape
    #reshape and tile function similar as that of the tensorflow
    x = F.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
    x = F.tile(x, [1, 1, 1, factor, 1, factor])
    x = F.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
    return x
    
#--------------------------------------------------------------


#----------------------------------------------------------------------
#downscale2d operation to be used 

def downscale2d(x, factor = 2, gain = 1):
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1

    if factor == 2:
        f = [np.sqrt(gain) / factor] * factor
        f = np.array(f, dtype = np.float32)
        if f.ndim == 1:
            f = f[:, np.newaxis] * f[np.newaxis, :]
        assert f.ndim == 2

        f = f[np.newaxis, :, :]
        f = np.tile(f, [int(x.shape[1]), 1, 1])
        w = nn.Variable.from_numpy_array(f)
        strides = (factor, factor)
        padding = (w.shape[1] // 2, w.shape[2] // 2)
        
        y = F.depthwise_convolution(x, w, stride = strides)
        return y 
    
    if gain != 1:
        x = x * gain

    if factor == 1:
        return x

    kernel = [factor, factor]
    y = F.average_pooling(x, kernel = kernel, stride = kernel)    # a bit of doubt regarding this statement
    return y

In [0]:
def minibatch_stddev_layer(x, group_size=4, num_new_features=1):
    group_size = min(group_size, x.shape[0])     # Minibatch must be divisible by (or smaller than) group_size.
    s = x.shape                                             # [NCHW]  Input shape.
    y = F.reshape(x, [group_size, -1, num_new_features, s[1]//num_new_features, s[2], s[3]])   # [GMncHW] Split minibatch into M groups of size G. Split channels into n channel groups c.
    y -= F.mean(y, axis=0, keepdims=True)           # [GMncHW] Subtract mean over group.
    y = F.mean(F.pow_scalar(y, 2.0), axis=0)                # [MncHW]  Calc variance over group.
    y = F.add_scalar(F.pow_scalar(y, 2.0) + 1e-8)                                   # [MncHW]  Calc stddev over group.
    y = F.mean(y, axis=[2,3,4], keepdims=True)      # [Mn111]  Take average over fmaps and pixels.
    y = F.mean(y, axis=[2])                         # [Mn11] Split channels into c channel groups
    y = F.tile(y, [group_size, 1, s[2], s[3]])             # [NnHW]  Replicate over group and pixels.
    return F.concatenate(x, y, axis=1)

In [0]:
def dense(x, fmaps, gain = np.sqrt(2), use_wscale = False, lrmul = 1, **kwargs):
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, lrmul, **kwargs)
    w = nn.parameter.get_parameter_or_create('Dense', shape = (x.shape[1], fmaps), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    return (F.affine(x, w, bias=None, base_axis=1))


def conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, **kwargs):
    assert isinstance(kernel, int) or isinstance(kernel, float)
    assert kernel >= 1 and kernel % 2 == 1
    if kernel == 1:
        padding = None
    else:
        padding = ((kernel - 2), (kernel - 2))  #tuple of padding
    kernel_tuple = (kernel, kernel)  #tuple of kernel_tuple
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    return PF.convolution(x, fmaps, kernel = kernel_tuple, w_init = w_init, with_bias = False, pad = padding, apply_w = lambda w: w * runtime_coeff)


def upscale2d_conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) * 2 >= 128

    if not fused_scale:
        return conv2d(upscale2d(x), fmaps, kernel, **kwargs)

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps, kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('deconv/W', shape = (x.shape[1], fmaps, kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2 , 2)
    return F.deconvolution(x, w, bias = None, pad = padding, stride = stride)


def conv2d_downscale2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    #assert kernel % 2 == 1 and kernel >= 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) >= 128   #why is this written and what is the condition behind that

    if not fused_scale:
        return downscale2d(conv2d(x, fmaps, kernel, **kwargs))

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)
    w_init, runtime_coeff = get_weight(he_std, gain = gain, use_wscale = use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('conv/W', shape = (fmaps, x.shape[1], kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    w = w * 0.25
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2,2)
    return F.convolution(x, w, bias=None, pad = padding, stride = stride)



In [0]:
#--------------------------------------------------------------
#Apply Bias
def apply_bias(x, lrmul = 1):
    b = nn.parameter.get_parameter_or_create(name = 'bias', shape = [x.shape[1]], initializer = I.ConstantInitializer(0))   
    b = b * lrmul
    if x.ndim == 2:
        print("DJ")
        return ((x + F.reshape(b, [1, -1])))
    return ((x + F.reshape(b, [1, -1, 1, 1]))) 

#--------------------------------------------------------------

In [0]:
#--------------------------------------
#application of the activation function
def leaky_relu(x):
    return functools.partial(F.leaky_relu, alpha = 0.2)(x)
#--------------------------------------

In [0]:
#---------------------------------------------
# defining the pixel normalization which is probably used in the Discriminator network
def pixel_norm(x, epsilon = 1e-8):
    mean1 = F.mean(F.pow_scalar(x, 2.0), axis = 1, keepdims = True)
    new_mean1 = F.add_scalar(mean1, epsilon)
    denominator = F.pow_scalar(new_mean1, 0.5)
    x = F.div2(x, denominator)
    return x

#--------------------------------------------------

In [0]:
#---------------------------------------------------------------
#Defining the Instance Normalization which is used in the Synthesis Generator Network with the format NCHW of the x   Change this FUnction i.e. Instance Normalization..
def instance_norm(x, epsilon = 1e-8):
    assert len(x.shape) == 4
    with nn.parameter_scope('InstanceNorm'):
        x1 = compute_channel_mean(x)
        x = x - x1
        x2 = compute_channel_mean(F.add_scalar(F.pow_scalar(x , 2.0), epsilon))
        x2 = F.pow_scalar(x2, 0.5)
        x = x / x2
        return x

#-----------------------------------------------------------------


In [0]:
#-------------------------------------------------------
#defining the style modulation i.e. combining ys and yb with the resultant of the addition of the noise and the feature maps
def style_mod(x, dlatent, **kwargs):
    with nn.parameter_scope('StyleMod'):
        style = apply_bias(dense(dlatent, fmaps = x.shape[1] * 2, **kwargs))
        style = F.reshape(style, [-1, 2, x.shape[1]] + [1] * (len(x.shape) - 2))
        return x * (style[:,0] + 1) + style[:,1]

#-------------------------------------------------------------------------------

In [0]:
#-------------------------------------------
#starting of the Mapping Network
def G_mapping(
    latents_in,  # First input: Latent vectors (Z)  [minibatch, latent_size]
    labels_in,   # Second input: Conditioning labels  [minibatch, label_size]
    latent_size = 512,   # Latent vector (Z) dimensionality
    label_size = 0,     # Label Dimensionality is 0, therefore no labels are present
    dlatent_size = 512, # Disentangled latent (W) dimensionality 
    dlatent_broadcast = 14,  # Output Disentangled latent (W) as [minibatch, dlatent_size] or [minibatch, dlatent_broadcast, dlatent_size]
    mapping_layers = 8,   # Number of Mapping Layers in the G_Mapping Network
    mapping_fmaps = 512,  # Number of activations in the mapping layers.
    mapping_lrmul = 0.01, # Learning rate multiplier for the mapping layers
    mapping_nonlinearity = 'lrelu',   # Activation function:  'relu', 'lrelu'
    use_wscale = True,    # Enable equalized Learning Rate
    normalize_latents = True,  # Normalizing the Latents before feeding them to the mapping Layers
    dtype = 'float32',  # Data type to use for activation and outputs
    **_kwargs
    ):
  
    act, gain = {'relu' : (F.relu, np.sqrt(2)), 'lrelu' : (leaky_relu, np.sqrt(2))}[mapping_nonlinearity]

    x = latents_in

    if label_size:
        with nn.parameter_scope('LabelConcat'):
            y = PF.affine(labels_in, latent_size)
            x = nn.concatenate(x, y, axis = 1)

    if normalize_latents:
        x = pixel_norm(x)


    with nn.parameter_scope('G_mapping'):
        for layer_idx in range(mapping_layers):
            scope_name = 'FC_{}'.format(layer_idx) 
            with nn.parameter_scope(scope_name):
                if layer_idx == mapping_layers:
                    famps = dlatent_size
                else:
                    fmaps = mapping_fmaps
                x = dense(x, fmaps, use_wscale=use_wscale, lrmul=mapping_lrmul)
                x = apply_bias(x, lrmul=mapping_lrmul)
                x = act(x)


    if dlatent_broadcast is not None:           
        with nn.parameter_scope('Broadcast'):
            x = F.tile(x[:, np.newaxis, :], [1, dlatent_broadcast, 1])

    return F.identity(x)

# End of the Mapping Network. 

In [0]:
#Using the Same Input as mentioned in above
z1 = latent_z[0]
z1 = nn.Variable.from_numpy_array(z1, need_grad=True)
print(z1.shape)

(1, 512)


In [0]:
 a1 = G_mapping(z1, None)

DJ
DJ
DJ
DJ
DJ
DJ
DJ
DJ


In [0]:
print(a1.shape)

(1, 14, 512)


In [0]:
#Getting the Parameters for Importing the Pre-Trained Weights
a2 = nn.get_parameters()

In [0]:
print(a2)

OrderedDict([('G_mapping/FC_0/Dense', <Variable((512, 512), need_grad=True) at 0x7f380096ff98>), ('G_mapping/FC_0/bias', <Variable((512,), need_grad=True) at 0x7f38008bc778>), ('G_mapping/FC_1/Dense', <Variable((512, 512), need_grad=True) at 0x7f38008bc728>), ('G_mapping/FC_1/bias', <Variable((512,), need_grad=True) at 0x7f38008bc6d8>), ('G_mapping/FC_2/Dense', <Variable((512, 512), need_grad=True) at 0x7f38008bc868>), ('G_mapping/FC_2/bias', <Variable((512,), need_grad=True) at 0x7f38008bc598>), ('G_mapping/FC_3/Dense', <Variable((512, 512), need_grad=True) at 0x7f38008bc908>), ('G_mapping/FC_3/bias', <Variable((512,), need_grad=True) at 0x7f38008bc818>), ('G_mapping/FC_4/Dense', <Variable((512, 512), need_grad=True) at 0x7f38008bc9a8>), ('G_mapping/FC_4/bias', <Variable((512,), need_grad=True) at 0x7f38008bc8b8>), ('G_mapping/FC_5/Dense', <Variable((512, 512), need_grad=True) at 0x7f38008bca48>), ('G_mapping/FC_5/bias', <Variable((512,), need_grad=True) at 0x7f38008bc958>), ('G_mappi

In [0]:
# Converting the Bias weights to Nnabla variabla
G_map_bias = []
for i in G_mapping_bias:
    l = nn.Variable.from_numpy_array(i)
    G_map_bias.append(l)

In [0]:
# Converting the Dense weights to Nnabla variabla
G_map_weights = []
for i in G_mapping_weights:
    l = nn.Variable.from_numpy_array(i)
    G_map_weights.append(l)

In [0]:
# Assigning the Pre-Trained weights to the Mapping Network
l = 0
l1 = 0
l2 = 0
for i, j in a2.items():
    print(i)
    if(l%2==0):
        j.data.copy_from(G_map_weights[l1].data)
        print("Value of l1 is:", l1)
        print("G_mapping weights shape is:", G_mapping_weights[l1].shape)
        print("J shape is:", j.shape)
        print()
        l1 = l1 + 1
    else:
        j.data.copy_from(G_map_bias[l2].data)
        print("Value of l2 is:", l2)
        print("G_mapping bias shape is:", G_mapping_bias[l2].shape)
        print("J shape is:", j.shape)
        print()
        l2 = l2 + 1
    l = l + 1

G_mapping/FC_0/Dense
Value of l1 is: 0
G_mapping weights shape is: (512, 512)
J shape is: (512, 512)

G_mapping/FC_0/bias
Value of l2 is: 0
G_mapping bias shape is: (512,)
J shape is: (512,)

G_mapping/FC_1/Dense
Value of l1 is: 1
G_mapping weights shape is: (512, 512)
J shape is: (512, 512)

G_mapping/FC_1/bias
Value of l2 is: 1
G_mapping bias shape is: (512,)
J shape is: (512,)

G_mapping/FC_2/Dense
Value of l1 is: 2
G_mapping weights shape is: (512, 512)
J shape is: (512, 512)

G_mapping/FC_2/bias
Value of l2 is: 2
G_mapping bias shape is: (512,)
J shape is: (512,)

G_mapping/FC_3/Dense
Value of l1 is: 3
G_mapping weights shape is: (512, 512)
J shape is: (512, 512)

G_mapping/FC_3/bias
Value of l2 is: 3
G_mapping bias shape is: (512,)
J shape is: (512,)

G_mapping/FC_4/Dense
Value of l1 is: 4
G_mapping weights shape is: (512, 512)
J shape is: (512, 512)

G_mapping/FC_4/bias
Value of l2 is: 4
G_mapping bias shape is: (512,)
J shape is: (512,)

G_mapping/FC_5/Dense
Value of l1 is: 5
G

In [0]:
l = 0
for i, j in a2.items():
    if(l==1):
        break
    l = l + 1
    print(i)
    print(j[0][0].d)
    print(j[0][1].d)
    print(j[0][2].d)
    print(j[0][3].d)
    print(j[0][4].d)
    print(j[0][5].d)
    print(j[0][6].d)
    print(j[0][7].d)

G_mapping/FC_0/Dense
-46.161606
-65.11477
74.91357
-46.699375
45.448704
-73.10281
-139.17113
-74.88173


In [0]:
a12 = G_mapping(z1, None)

DJ
DJ
DJ
DJ
DJ
DJ
DJ
DJ


In [0]:
print(a12.shape)

(1, 14, 512)


In [0]:
#Printing the First 10 values of the Output
print(a12[0][0][0].d)
print(a12[0][0][1].d)
print(a12[0][0][2].d)
print(a12[0][0][3].d)
print(a12[0][0][4].d)
print(a12[0][0][5].d)
print(a12[0][0][6].d)
print(a12[0][0][7].d)
print(a12[0][0][8].d)
print(a12[0][0][9].d)

0.005208131
-0.16036135
-0.017817566
0.0980893
-0.03497812
0.38442647
1.3938818
-0.065430276
0.049763903
-0.07020566


In [0]:
#Printing the Entire Output
print(a12.d)

[[[ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]
  [ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]
  [ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]
  ...
  [ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]
  [ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]
  [ 0.00520813 -0.16036135 -0.01781757 ... -0.09900362 -0.18152945
   -0.21427055]]]
